## <center><i>Muestreo aleatorio (ficticio) de subcuencas</i></center>
---

In [2]:
import shapely 
import geemap.foliumap as geemap
import ipywidgets as widgets
import leafmap.foliumap as leafmap
import geopandas as gpd
import ee

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1ATx3LY6Ou4aK0G49idVt6o2VWE70l813bszxOnGgCnnP2eXA0Ji3Xwqpwo4



Successfully saved authorization token.


In [4]:
suhs = ee.FeatureCollection("users/jorgeluiscruzquispe7/SubUnidadesHidrograficas")

suhs_gdf = geemap.ee_to_gdf(suhs)

dropdown = widgets.Dropdown(options = suhs_gdf['Nombre_UH'].tolist(), 
                            value = "Cuenca Gorgor", 
                            description = 'Selecciona un valor: ')

dropdown

Dropdown(description='Selecciona un valor: ', index=47, options=('Cuenca Unidad Hidrografica 131536', 'Cuenca …

In [5]:
subcuenca = ee.FeatureCollection("users/jorgeluiscruzquispe7/SubUnidadesHidrograficas")\
                                            .filter(ee.Filter.eq("Nombre_UH", dropdown.value))

red_hidrografica = ee.FeatureCollection("users/jorgeluiscruzquispe7/red_hidrografica_cenepred").filterBounds(subcuenca)

subcuenca_gdf = geemap.ee_to_gdf(subcuenca)

red_gdf = geemap.ee_to_gdf(red_hidrografica)

Map = leafmap.Map()

Map.add_basemap("HYBRID")

Map.add_gdf(subcuenca_gdf, style = {"color": "red", "fillOpacity": 0}, layer_name = "layer subcuencas")

Map.add_gdf(red_gdf, style = {"color": "blue", "fillOpacity": 0}, layer_name = "red hidrografica")

Map

In [6]:
points = geemap.ee_to_gdf(ee.FeatureCollection.randomPoints(region = subcuenca, 
                                                            points = 100, # cantidad de puntos
                                                            seed = 42, # la semilla considera la aleatoriedad
                                                           ))

points.crs = "EPSG:4326"

points["Tipo"] = "punto"

points[f"Número ({(len(points))})"] = 0

points["Latitud"] = round(points.bounds.iloc[:,1], 4)

points["Longitud"] = round(points.bounds.iloc[:,0], 4)

points.to_crs(32718, inplace = True)

for i in range(len(points)):
    points.loc[i, f"Número ({(len(points))})"] = i + 1
    
Map = leafmap.Map()

Map.add_basemap("HYBRID")

Map.add_gdf(subcuenca_gdf, style = {"color": "red", "fillOpacity": 0}, layer_name = "layer subcuencas")

Map.add_points_from_xy(points, x = "Longitud", y = "Latitud", marker_colors = ["orange"], icon_names = ["star"],
                       color_column = "Tipo", layer_name = "puntos")

Map

In [7]:
points_ = points.copy()

distance = 100 # se considera rectangulos  de 100 x 100

points_["puntos1"] = points_['geometry'].apply(lambda punto: shapely.geometry.Point(punto.x + distance, punto.y))

points_["puntos2"] = points_['geometry'].apply(lambda punto: shapely.geometry.Point(punto.x, punto.y + distance))

points_["puntos3"] = points_['geometry'].apply(lambda punto: shapely.geometry.Point(punto.x + distance, punto.y + distance))

points_['multipoint'] = points_.apply(lambda point: shapely.geometry.Polygon([
    point['geometry'], point['puntos1'], point['puntos3'], point['puntos2'],
]), axis = 1)

gdf_multipoint_utm = gpd.GeoDataFrame(geometry = points_['multipoint'], crs = "EPSG:32718")

gdf_multipoint_utm["Area (m2)"] = round(gdf_multipoint_utm.area, 3)

gdf_multipoint_wgs = gdf_multipoint_utm.to_crs(4326)

In [8]:
Map = leafmap.Map()

Map.add_basemap("HYBRID")

Map.add_gdf(subcuenca_gdf, style = {"color": "red", "fillOpacity": 0}, layer_name = "layer subcuencas")

Map.add_gdf(gdf_multipoint_wgs, style = {"color": "blue", "fillOpacity": 0}, layer_name = "areas")

Map

In [9]:
df_parcela = gdf_multipoint_utm.centroid.to_frame().rename(columns = {0: "centroide_parcela"})

df_parcela = df_parcela.centroide_parcela.bounds.drop(columns = ["maxx", "maxy"])\
                        .rename(columns = {"minx": "Longitud", "miny": "Latitud"})

df_parcela

,Longitud,Latitud
0,281881.119573,8.830707e+06
1,282733.257795,8.824979e+06
2,294013.944611,8.824971e+06
3,278337.218242,8.828991e+06
4,269144.685418,8.832977e+06
...,...,...
95,262295.074668,8.833844e+06
96,288215.671435,8.833297e+06
97,282215.182201,8.827342e+06
98,268715.905868,8.820580e+06


In [10]:
df_parcela.to_csv("centroides_parcelas.csv")